# Summary

The following code in this notebook demonstrates my exploration of the three datasets (Product, Transaction, and User) to identify data quality issues.

It also includes SQL queries tailored to answer both closed-ended and open-ended questions outlined in the data challenge, while highlighting an interesting data trend observed in the provided datasets.

# Import Packages & Functions

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# utility function import
from util import *

In [2]:
product = pd.read_csv('PRODUCTS_TAKEHOME.csv')
transaction = pd.read_csv('TRANSACTION_TAKEHOME.csv')
user = pd.read_csv('USER_TAKEHOME.csv')

In [3]:
product.name = 'product'
transaction.name = 'transaction'
user.name = 'user'

# Product Table Analysis

### Key Data Quality Issues in the Product Table:

1. **Missing Values**:
   - Several columns have a significant percentage of missing data:
     - **CATEGORY_4**: 92.02% missing, which suggests that product categorization is incomplete or not consistently recorded for most of the products.
     - **MANUFACTURER** and **BRAND**: Both have 26.78% missing values, which could imply inconsistent manufacturer and brand records.
     - **CATEGORY_3**: 7.16% missing, indicating some level of missing categorization in this field.
     - **BARCODE**: While missing values in the `BARCODE` column are relatively low (0.48%), it still poses an issue for uniqueness and data integrity.

2. **Duplicate Values**:
   - **BARCODE** contains duplicate values (2 duplicates), even though it’s expected to be a unique identifier. This suggests that some product entries share the same barcode, which violates the uniqueness requirement for primary keys.
   - **57 duplicate rows** exist in the product table, which could lead to data inconsistencies and redundancy in reporting or analysis.

3. **Mixed Data Types**:
   - The **MANUFACTURER** column contains both numeric and string values, which complicates analysis and standardization. These mixed data types should be unified to maintain consistency across the column, likely converting numeric values to strings.

4. **Potential Unique Identifiers**:
   - The `BARCODE` column, which should be a unique identifier, has missing values and duplicates, making it unreliable as a primary key without further data cleaning.

5. **Data Type Overview**:
   - Some columns, such as **CATEGORY_4**, **MANUFACTURER**, and **BRAND**, have large proportions of missing values, which suggests that these fields may not be fully utilized or correctly filled.
   - **Unique values**: There are large numbers of unique values in columns like **MANUFACTURER** (4354 unique values), which might require standardization or grouping for better consistency in reporting.

6. **Null Column Analysis**:
   - The **CATEGORY_4** column has a very high percentage of missing values, which could indicate that this field is not being used or recorded for a large portion of the products.

### Challenges to Understand:
1. **Missing Values Impact**:
   - A large proportion of missing values in **CATEGORY_4**, **MANUFACTURER**, and **BRAND** suggests potential gaps in the data collection process. It will be important to assess why these fields are missing so frequently and whether it’s due to data entry issues, inconsistent reporting, or irrelevant for some products.

2. **Duplicate Barcodes**:
   - The **BARCODE** field, which is expected to be a unique identifier, has duplicate values. Understanding why there are duplicates in this field and resolving them is crucial for ensuring data integrity, as barcodes should typically be unique for each product.

3. **Mixed Data Types**:
   - The **MANUFACTURER** column contains both numeric and string data types, which could complicate further analysis. Deciding how to handle mixed data types and standardizing this field is important for consistency.

4. **Categorization Inconsistencies**:
   - The **CATEGORY_4** column has a significant amount of missing data. This suggests that either the categorization is incomplete or inconsistent, and the overall data structure could be impacted. Clarifying whether this column is necessary or if missing data should be handled differently will be important.

5. **Data Redundancy**:
   - The 57 duplicate rows need to be removed to maintain data consistency and ensure accurate analysis.

In [4]:
product.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHANGEABLY.,MAPLE HOLISTICS,8.068109e+11


Notes:

- The `BARCODE` field will lose leading zeros if stored as a numeric data type, which may compromise data integrity. To ensure accurate representation and maintain consistency, it should be stored as a string.

In [5]:
# Stored the BARCODE datatype to string
product = pd.read_csv('PRODUCTS_TAKEHOME.csv', dtype={'BARCODE':str})

product.name = 'product'

## Product Table Overview

In [6]:
print(print_summary(product))


    Here is the result of the initial data check for of table 'product':

    1. Basic Table Information:
       - DataFrame has 845552 rows and 7 columns.
    
    2. Column Name Issues:
       - No column names have extra spaces.
       - No duplicate column names.

    3. Data Type Overview:
             Column Data Type  Null Values  Unique Values
  CATEGORY_4    object       778093            127
MANUFACTURER    object       226474           4354
       BRAND    object       226472           8122
  CATEGORY_3    object        60566            344
     BARCODE    object         4025         841525
  CATEGORY_2    object         1424            121
  CATEGORY_1    object          111             27
    
    4. Potential Unique Identifiers in product table:
       - No unique and non-null identifier detected.
       - Invalid unique identifiers: BARCODE in product table has 4025 missing values (NaN)., BARCODE in product table has 2 duplicate values (excluding NaN).
    
    5. Null 

## Data Cleaning

**To Do Next**:

1. **Unique Identifiers - BARCODE**: The `BARCODE` field is expected to be a unique identifier and could serve as the primary key. However, it contains both 4025 null values and 2 duplicate values. To maintain its integrity as a unique identifier, **consider removing the duplicates and addressing the null values**.

2. **Missing Values**: The `CATEGORY_4` field is missing data in over 90% of records, indicating that this level of product categorization may be incomplete or inconsistently recorded. **Consider excluding this column from the dataset**.

3. **Duplicate Rows**: There are 57 duplicate rows in the product table. **Consider removing the duplicate rows to ensure data consistency**.

4. **Data Type Consistency - MANUFACTURER**: The `MANUFACTURER` field contains both numeric and string data types. To standardize the data, **consider converting the numeric values to string format**, or **removing the numeric values from the field**, ensuring consistency across the column.

### Remove duplicate BARCODE

In [7]:
# Get duplicate values in the 'BARCODE' column, excluding NaN values
duplicate_barcodes = product[product['BARCODE'].notna() & product['BARCODE'].duplicated(keep=False)]

# Display the duplicate BARCODE values
duplicate_barcodes

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
28421,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,052336919068
213340,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,017000329260
304021,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,017000329260
709607,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,052336919068


Notes:
- The duplicate BARCODE values are 017000329260 and 052336919068. **Remove these duplicate barcodes** to maintain data integrity.

In [8]:
# Remove rows where 'BARCODE' is '052336919068' or '017000329260'

duplicate_barcodes = ['052336919068', '017000329260']
product = product[~product['BARCODE'].isin(duplicate_barcodes)]

### Remove null BARCODE

In [9]:
# Get null values in the 'BARCODE' column
null_barcodes = product[product['BARCODE'].isna()]

# Display the null BARCODE values
null_barcodes.head(10)

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
9,Health & Wellness,NaN,NaN,NaN,CHURCH & DWIGHT,REPHRESH,NaN
124,Snacks,Jerky & Dried Meat,NaN,NaN,CONAGRA,SLIM JIM,NaN
147,Snacks,Crackers,Wheat Crackers,NaN,KELLANOVA,CARR'S,NaN
365,Health & Wellness,Hair Care,Shampoo,NaN,UNILEVER,APOTHECARE ESSENTIALS,NaN
428,Health & Wellness,Eye Care,Contact Lens Solution,NaN,BAUSCH + LOMB CORPORATION,BAUSCH + LOMB,NaN
670,Health & Wellness,Bath & Body,Hand & Body Lotions,NaN,COTY,MARC JACOBS FRAGRANCE & BODY CARE,NaN
699,Health & Wellness,Bath & Body,NaN,NaN,SANOFI AVENTIS US LLC,GOLD BOND,NaN
838,Health & Wellness,NaN,NaN,NaN,FRISKA,FRISKA,NaN
1840,Health & Wellness,Bath & Body,Hand Sanitizer Wipes,NaN,GEORGIA-PACIFIC,ANGEL SOFT,NaN
2075,Health & Wellness,Skin Care,NaN,NaN,"BEIERSDORF, INC.",COPPERTONE,NaN


In [10]:
# Remove rows where BARCODE is NaN

product.dropna(subset=['BARCODE'], inplace=True)

### Remove Duplicate Rows

In [11]:
# Remove duplicate rows, retaining the first occurrence.

product.drop_duplicates(keep='first', inplace=True)

### Standardize MANUFACTURER data types

In [12]:
# Check for numeric values in the 'MANUFACTURER' column
numeric_manufacturer = product[pd.to_numeric(product['MANUFACTURER'], errors='coerce').notna()]

# Display the results
numeric_manufacturer.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
38678,Health & Wellness,Foot Care,Insoles & Orthotics,NaN,5.11,5.11 TACTICAL,888579041750
98098,Health & Wellness,Foot Care,Insoles & Orthotics,NaN,5.11,5.11 TACTICAL,888579041606
118667,Health & Wellness,Foot Care,Insoles & Orthotics,NaN,5.11,5.11 TACTICAL,888579034523
132984,Health & Wellness,Foot Care,Insoles & Orthotics,NaN,5.11,5.11 TACTICAL,888579041514
149331,Health & Wellness,Foot Care,Insoles & Orthotics,NaN,5.11,5.11 TACTICAL,888579041637


Notes:

- It turns out that 5.11 is a valid manufacturer name. **Convert it to a string to ensure it is stored correctly**.

In [13]:
# Convert numeric values in the 'MANUFACTURER' column to strings

product['MANUFACTURER'] = product['MANUFACTURER'].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)

# Transaction Table Analysis

### Key Data Quality Issues in the Transaction Table:

**Missing Values:**
- **BARCODE**: Contains **11.52% missing values** (5762 records), which may impact transaction mapping and product identification.
- **FINAL_QUANTITY** and **FINAL_SALE**: Both fields have a mix of numeric and string values, and missing entries could affect overall data analysis, particularly in transaction totals.

**Duplicate Values:**
- **RECEIPT_ID**: There are **25,560 duplicate values** (excluding `NaN`), which undermine its effectiveness as a unique identifier for transactions.
- **171 duplicate rows** exist in the table, leading to potential data inconsistencies and redundancy in reporting and analysis.

**Mixed Data Types:**
- **USER_ID**: Contains both **string** and **numeric** values, which may complicate analysis and reporting.
- **FINAL_QUANTITY** and **FINAL_SALE**: These fields contain both **string** and **numeric** data types (e.g., the `FINAL_QUANTITY` column has string entries like 'ZERO' instead of numeric `0`), which need to be standardized for consistency.

**Potential Unique Identifiers:**
- **RECEIPT_ID**: Although it should serve as a unique identifier, it contains duplicates, making it unreliable for uniquely identifying transactions without further data cleaning.

**Date Order Issues:**
- **SCAN_DATE**: There are **94 rows** where **`scan_date`** is before **`purchase_date`**, which could lead to incorrect transaction data or impact the accuracy of time-based reporting.

**Data Type Overview:**
- **USER_ID**, **FINAL_QUANTITY**, and **FINAL_SALE**: These columns contain inconsistent data types (e.g., strings mixed with numeric values), which need to be standardized for consistent reporting.



### Challenges to Understand:

**Missing Values Impact:**
- The **BARCODE** field has a significant proportion of missing values, which may affect transaction mapping accuracy.
- Missing values in **FINAL_QUANTITY** and **FINAL_SALE** could distort transaction calculations, such as total sales or quantity sold. Understanding why these fields have missing values and filling or handling them appropriately is essential.

**Duplicate RECEIPT_ID Entries:**
- **RECEIPT_ID** is expected to be a unique identifier, but the presence of **25,560 duplicates** suggests that this field is not functioning as intended. Resolving this issue will be critical for ensuring data integrity and the accurate tracking of transactions.

**Mixed Data Types:**
- The **USER_ID**, **FINAL_QUANTITY**, and **FINAL_SALE** fields contain both **string** and **numeric** values, which will complicate analysis and reporting. A decision must be made on how to standardize these fields (e.g., converting strings to numbers where appropriate).

**Date Order Issues:**
- **SCAN_DATE** values occurring before **PURCHASE_DATE** need to be investigated further. This is critical to ensure that the transaction timeline is correct, especially for time-sensitive analyses or reporting.

**Categorization Inconsistencies:**
- Inconsistent or missing values in **FINAL_QUANTITY** and **FINAL_SALE** fields, especially with string values such as 'ZERO', point to the need for standardization and consistent data entry practices. Clarifying whether these fields should contain only numeric values and cleaning the data is necessary.

**Data Redundancy:**
- The **171 duplicate rows** must be removed to ensure that each transaction is only represented once in the data. This will help prevent skewed analysis and improve the reliability of any insights derived from the dataset.

In [14]:
transaction.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


Notes: 

- Similar to the `BARCODE` field in the product table, which is stored as a string to preserve leading zeros, this approach ensures no data is lost when leading zeros are significant, as would happen if stored as a numeric data type.


In [15]:
# Stored the BARCODE datatype to string
transaction = pd.read_csv('TRANSACTION_TAKEHOME.csv', dtype={'BARCODE':str})

transaction.name = 'transaction'

In [16]:
# Convert 'PURCHASE_DATE' and 'SCAN_DATE' to datetime objects
transaction['PURCHASE_DATE'] = pd.to_datetime(transaction['PURCHASE_DATE'], utc=True, errors='coerce')
transaction['SCAN_DATE'] = pd.to_datetime(transaction['SCAN_DATE'], utc=True, errors='coerce')

## Transaction Table Overview

In [17]:
print(print_summary(transaction))


    Here is the result of the initial data check for of table 'transaction':

    1. Basic Table Information:
       - DataFrame has 50000 rows and 8 columns.
    
    2. Column Name Issues:
       - No column names have extra spaces.
       - No duplicate column names.

    3. Data Type Overview:
               Column           Data Type  Null Values  Unique Values
       BARCODE              object         5762          11028
    RECEIPT_ID              object            0          24440
     SCAN_DATE datetime64[ns, UTC]            0          24440
 PURCHASE_DATE datetime64[ns, UTC]            0             89
    STORE_NAME              object            0            954
       USER_ID              object            0          17694
FINAL_QUANTITY              object            0             87
    FINAL_SALE              object            0           1435
    
    4. Potential Unique Identifiers in transaction table:
       - No unique and non-null identifier detected.
       - I

## Data Cleaning

**To Do Next**:

1. **Unique Identifiers - RECEIPT_ID**:  
The `RECEIPT_ID` field is expected to be a unique identifier and could serve as the primary key. However, it contains 25,560 duplicate values. Further investigation is **needed to understand how receipt items are listed in the transaction table** and to determine whether the duplicate `RECEIPT_ID` values should be retained.

3. **Missing Values - BARCODE**:  
The `BARCODE` field is missing data in 12% of records. Since `BARCODE` serves as a foreign key to connect the Product table, **consider removing records with missing BARCODE values** to ensure accurate transaction mapping and reliable brand/manufacturer trend analysis.

5. **Duplicate Rows**:  
There are 171 duplicate rows in the product table. **Consider removing the duplicate rows** to ensure data consistency.

7. **Data Type Consistency - USER_ID, FINAL_QUANTITY, FINAL_SALE**:  
The `USER_ID`, `FINAL_QUANTITY`, `FINAL_SALE` fields contain both numeric and string data types. To standardize the data, **consider converting the numeric values to string format**, or **removing the numeric / string values from the field**, ensuring consistency across the column.

9. **Data Outliers - FINAL_QUANTITY, FINAL_SALE**:  
Convert the `FINAL_QUANTITY` and `FINAL_SALE` fields to numeric values and check for any abnormal data or outliers.

11. **Data Accuracy - PURCHASE_DATE, SCAN_DATE**:  
There exists 94 receipts where the `SCAN_DATE` **precedes** the `PURCHASE_DATE`, indicating data discrepancies. As a temporary solution, these **records should be removed** to maintain data consistency.

### Investigate on RECEIPT_ID & Standardize FINAL_QUANTITY and FINAL_SALE data types

In [18]:
# Get duplicate values in the 'RECEIPT_ID' column

transaction[transaction['RECEIPT_ID'].duplicated(keep=False)].sort_values(by='RECEIPT_ID').head(10)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21 00:00:00+00:00,2024-08-21 14:19:06.539000+00:00,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.00,
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21 00:00:00+00:00,2024-08-21 14:19:06.539000+00:00,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.00,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20 00:00:00+00:00,2024-07-20 09:50:24.206000+00:00,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
39291,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20 00:00:00+00:00,2024-07-20 09:50:24.206000+00:00,ALDI,62c08877baa38d1a1f6c211a,NaN,1.00,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18 00:00:00+00:00,2024-08-19 15:38:56.813000+00:00,WALMART,60842f207ac8b7729e472020,078742229751,1.00,
25928,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18 00:00:00+00:00,2024-08-19 15:38:56.813000+00:00,WALMART,60842f207ac8b7729e472020,078742229751,1.00,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18 00:00:00+00:00,2024-06-19 11:03:37.468000+00:00,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,zero,3.49
41475,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18 00:00:00+00:00,2024-06-19 11:03:37.468000+00:00,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,1.00,3.49
43233,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04 00:00:00+00:00,2024-07-05 15:56:43.549000+00:00,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.00,5.29
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04 00:00:00+00:00,2024-07-05 15:56:43.549000+00:00,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.00,


Notes:
- **Duplicate `RECEIPT_ID` Entries**: Purchased items are recorded twice under the same `RECEIPT_ID`, but there are missing values in either the **final quantity** or **final sale** fields.

    - For example, for `RECEIPT_ID` **0000d256-4041-4a3e-adc4-5623fb6e0c99**, two items purchased on **08/21/2024** from **Walmart** are recorded. The barcodes are the same, but one entry has a **final sale** of **1.54**, while the other has an empty value.

- **Inconsistent Data Types**: There are inconsistencies in the data types within the **final quantity** and **final sale** fields.

    - For example, the **final quantity** field contains the string `'ZERO'` instead of the integer `0`.


Solutions:

- Standardize the `FINAL_QUANTITY` and `FINAL_SALE` data types first. 
- To resolve this, **standardize the values for the duplicate `RECEIPT_ID` entries**. If there are discrepancies in the **final sale** or **final quantity** fields, **consider to fill in the missing values with the maximum values from either the final_sale or final_quantity field**.
- After standardizing, **remove the duplicate entries** to ensure that only one record per purchase remains.

#### Standardize FINAL_QUANTITY and FINAL_SALE data types

In [19]:
# check what strings include in FINAL_QUANTITY field
transaction['FINAL_QUANTITY'].unique()

array(['1.00', 'zero', '2.00', '3.00', '4.00', '4.55', '2.83', '2.34',
       '0.46', '7.00', '18.00', '12.00', '5.00', '2.17', '0.23', '8.00',
       '1.35', '0.09', '2.58', '1.47', '16.00', '0.62', '1.24', '1.40',
       '0.51', '0.53', '1.69', '6.00', '2.39', '2.60', '10.00', '0.86',
       '1.54', '1.88', '2.93', '1.28', '0.65', '2.89', '1.44', '2.75',
       '1.81', '276.00', '0.87', '2.10', '3.33', '2.54', '2.20', '1.93',
       '1.34', '1.13', '2.19', '0.83', '2.61', '0.28', '1.50', '0.97',
       '0.24', '1.18', '6.22', '1.22', '1.23', '2.57', '1.07', '2.11',
       '0.48', '9.00', '3.11', '1.08', '5.53', '1.89', '0.01', '2.18',
       '1.99', '0.04', '2.25', '1.37', '3.02', '0.35', '0.99', '1.80',
       '3.24', '0.94', '2.04', '3.69', '0.70', '2.52', '2.27'],
      dtype=object)

Note: 
- The `final quantity` field contains the string 'ZERO' instead of the integer 0. **Consider converting to a numeric format** to enable accurate quantity calculations.

In [20]:
# Replace 'zero' with 0.00 and convert to float
transaction['FINAL_QUANTITY'] = transaction['FINAL_QUANTITY'].replace('zero', '0.00')
transaction['FINAL_QUANTITY'] = transaction['FINAL_QUANTITY'].astype(float)

In [21]:
# check what strings include in FINAL_QUANTITY field
transaction['FINAL_SALE'].unique()

array([' ', '1.49', '3.49', ..., '11.02', '20.17', '42.38'],
      shape=(1435,), dtype=object)

Note: 
- The `final sale` field contains empty spaces instead of integer 0. **Consider converting these empty spaces to integer 0** to standardize the data.

In [22]:
# Replace 'zero' with 0.00 and convert to float
transaction['FINAL_SALE'] = transaction['FINAL_SALE'].replace(' ', '0.00')
transaction['FINAL_SALE'] = transaction['FINAL_SALE'].astype(float)

#### Check FINAL_QUANTITY and FINAL_SALE data range

In [23]:
transaction.describe(include=np.number)

,FINAL_QUANTITY,FINAL_SALE
count,50000.000000,50000.000000
mean,0.814427,3.432288
std,1.841794,6.070526
min,0.000000,0.000000
25%,0.007500,0.000000
50%,1.000000,2.180000
75%,1.000000,4.440000
max,276.000000,462.820000


Note: 
- An outlier receipt with an unusually high `FINAL_QUANTITY` value of 276 and `FINAL_SALE` value of 462.82 was identified. Further investigation is needed to **determine whether to remove these potentially unrealistic data points**.

In [24]:
# Check the receipt records with final_quantity = 276 and final_sale = 462.82

transaction[(transaction['FINAL_QUANTITY'] == 276) | (transaction['FINAL_SALE'] == 462.820000)]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
13617,8bbb09f6-aae6-47ae-88af-7cd46cc8079d,2024-07-06 00:00:00+00:00,2024-07-06 15:01:20.305000+00:00,CVS,630789e1101ae272a4852287,NaN,0.0,462.82
24810,fe0780d1-2d02-4822-8f12-7056b1814f17,2024-08-09 00:00:00+00:00,2024-08-11 17:52:18.523000+00:00,MAIN STREET MARKET,5d197f9dd08976510c49d0e6,048001353664,276.0,0.00
26692,8bbb09f6-aae6-47ae-88af-7cd46cc8079d,2024-07-06 00:00:00+00:00,2024-07-06 15:01:20.305000+00:00,CVS,630789e1101ae272a4852287,NaN,1.0,462.82
42410,fe0780d1-2d02-4822-8f12-7056b1814f17,2024-08-09 00:00:00+00:00,2024-08-11 17:52:18.523000+00:00,MAIN STREET MARKET,5d197f9dd08976510c49d0e6,048001353664,276.0,5.89


Note:
- It seems unrealistic to buy **276 items** with a **final sale** of **5.89**, or purchase **one item from CVS** with a **final sale** of **462.82**. Therefore, records with **`FINAL_QUANTITY` = 276** and **`FINAL_SALE` = 462.82** will be removed to maintain data accuracy.

In [25]:
#  # Remove records where FINAL_QUANTITY = 276 and FINAL_SALE = 462.82

transaction = transaction[~((transaction['FINAL_QUANTITY'] == 276) & (transaction['FINAL_SALE'] == 462.820000))]

#### Merge duplicate RECEIPT_ID 

Merge the duplicate `receipt_id` entries by filling the missing values with the **maximum value** from either the `final_sale` or `final_quantity` field.

In [26]:
# Group by all columns except FINAL_QUANTITY and FINAL_SALE
grouped = transaction.groupby(['RECEIPT_ID', 'PURCHASE_DATE', 'SCAN_DATE', 'STORE_NAME', 'USER_ID', 'BARCODE'])

# Apply aggregation to handle duplicate receipts
transaction = grouped.agg(
    {
        'FINAL_QUANTITY': 'max',  # Keep the maximum FINAL_QUANTITY
        'FINAL_SALE': 'max'       # Keep the maximum FINAL_SALE
    }
).reset_index()

### Standardize USER_ID data types

In [27]:
# Check for numeric values in the 'USER_ID' column

transaction[pd.to_numeric(transaction['USER_ID'], errors='coerce').notna()]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
1562,123e4e14-96ba-4e48-88ba-01c40a7f8a63,2024-06-24 00:00:00+00:00,2024-07-01 13:46:16.242000+00:00,WALMART,635332771020216633290897,051500255162,1.0,3.12
21633,fbfc5143-2b6e-4045-8c41-73cb530b4e78,2024-09-01 00:00:00+00:00,2024-09-01 21:59:19.874000+00:00,SAM'S CLUB,6211728204717e7604896941,030000561959,1.0,10.98


Note: 
- The USER_ID values are already in the correct numeric format. Leave them as they are.

### Remove null BARCODE

In [28]:
# Remove rows where 'BARCODE' is null in the transaction dataframe

transaction.dropna(subset=['BARCODE'], inplace=True)

### Remove Duplicate Rows 

In [29]:
# Remove duplicate rows, retaining the first occurrence.

transaction.drop_duplicates(keep='first', inplace=True)

### Remove SCAN_DATE before PURCHASE_DATE

In [30]:
# Drop the records where 'SCAN_DATE' is later than 'PURCHASE_DATE'

transaction = transaction[transaction['PURCHASE_DATE'].dt.date <= transaction['SCAN_DATE'].dt.date]
transaction.name = 'transaction'

# User Table Analysis

### Key Data Quality Issues in the User Table:

- **Missing Values**:
- The **LANGUAGE**, **GENDER**, **STATE**, and **BIRTH_DATE** fields contain significant missing values, especially **LANGUAGE** (30.51%). It's important to assess the impact of these missing values on the overall analysis.
  
  **Note**: Since these fields are not critical as key identifiers, it is acceptable to leave them as is for now.

**Date Validity and Consistency**:
- **CREATED_DATE vs. BIRTH_DATE**: There is one record where **`CREATED_DATE`** is earlier than **`BIRTH_DATE`**, which is invalid. This could indicate an error in the data entry process.

**Data Consistency**:
- **GENDER**: The `GENDER` field contains multiple variations in responses (e.g., "non_binary" vs. "Non-Binary" and "not_specified"). This inconsistency makes it difficult to categorize and analyze gender data properly. Standardizing the gender values and consolidating similar responses into a consistent set is required.


### Challenges to Understand:

**Missing Data Impact**:
- Although the **LANGUAGE**, **GENDER**, **STATE**, and **BIRTH_DATE** fields are not key fields, addressing missing data for **LANGUAGE** and **GENDER** may improve user profiling.


**Date Anomalies**:
- The existence of one record where **CREATED_DATE** is earlier than **BIRTH_DATE** presents an issue that needs investigation. Such anomalies should be corrected to ensure data accuracy, as this inconsistency may affect further analysis.

In [31]:
user.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


In [32]:
# Convert 'CREATED_DATE' and 'BIRTH_DATE' to datetime objects
user['CREATED_DATE'] = pd.to_datetime(user['CREATED_DATE'], utc=True, errors='coerce')
user['BIRTH_DATE'] = pd.to_datetime(user['BIRTH_DATE'], utc=True, errors='coerce')

## User Table Overview

In [33]:
print(print_summary(user))


    Here is the result of the initial data check for of table 'user':

    1. Basic Table Information:
       - DataFrame has 100000 rows and 6 columns.
    
    2. Column Name Issues:
       - No column names have extra spaces.
       - No duplicate column names.

    3. Data Type Overview:
             Column           Data Type  Null Values  Unique Values
    LANGUAGE              object        30508              2
      GENDER              object         5892             11
       STATE              object         4812             52
  BIRTH_DATE datetime64[ns, UTC]         3675          54721
CREATED_DATE datetime64[ns, UTC]            0          99942
          ID              object            0         100000
    
    4. Potential Unique Identifiers in user table:
       - Potential unique identifier(s): ['ID'].
    
    5. Null Column Analysis:
       - Partially NULL columns: 
          - LANGUAGE: 30508 missing (30.51%)
          - GENDER: 5892 missing (5.89%)
          - S

## Data Cleaning

**To Do Next**: 

1. **Missing Values - LANGUAGE, GENDER, STATE, BIRTH_DATE**:  
The `LANGUAGE` field has **30.51% missing data**, `GENDER` has **5.89% missing**, `STATE` has **4.81% missing**, and `BIRTH_DATE` has **3.68% missing**. Since these fields are **not key fields**, it is **acceptable to leave them as is for now**.

3. **Data Type Consistency - ID**:  
The `ID` field contains mixed data types (string and numeric). To ensure consistency across the column, **consider standardizing the data type by converting all values to a single format (either string or numeric)**.

4. **Date Order - CREATED_DATE vs. BIRTH_DATE**:  
There is **1 row** where `CREATED_DATE` is **earlier** than `BIRTH_DATE`, which is inconsistent with the expected data behavior. **Investigate and correct this data anomaly** by removing or correcting the affected row.


8. **Data Accuracy - LANGUAGE, GENDER and STATE**:  
The `GENDER` field contains variations in responses (e.g., "non_binary" vs. "Non-Binary"), which leads to inconsistencies in the data. Standardize the gender values by ensuring uniform formatting and consolidating similar responses into a consistent set of categories.


### Standardize ID data types

In [34]:
# Check for numeric values in the 'ID' column

user[pd.to_numeric(user['ID'], errors='coerce').notna()]

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
58624,635410041020216633297518,2022-10-22 15:45:08+00:00,2002-03-30 05:00:00+00:00,MI,en,female


Note: 
- The USER_ID values are already in the correct numeric format. Leave them as they are.

### Remove CREATE_DATE before BIRTH_DATE

In [35]:
# Drop the records where 'SCAN_DATE' is later than 'PURCHASE_DATE'

user = user[user['CREATED_DATE'].dt.date >= user['BIRTH_DATE'].dt.date]
user.name = 'user'

### Data Consistency in GENDER 

In [36]:
user['GENDER'].unique()

array(['female', 'male', 'non_binary', 'transgender', nan,
       'prefer_not_to_say', 'not_listed', 'Non-Binary', 'unknown',
       'not_specified', "My gender isn't listed", 'Prefer not to say'],
      dtype=object)

In [37]:
# Replace values in the 'GENDER' column
user['GENDER'] = user['GENDER'].replace({'Non-Binary': 'non_binary',
                                         'Prefer not to say': 'prefer_not_to_say',
                                         'My gender isn\'t listed': 'not_listed',
                                         'not_specified': 'prefer_not_to_say'})

# Data Analysis by SQL

In [38]:
import sqlite3

# Establish a connection to the SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect('fetch_data_challenge.db')

cursor = conn.cursor()

# 'product', 'transaction', and 'user' are your pandas DataFrames
product.to_sql('product', conn, if_exists='replace', index=False)
transaction.to_sql('transaction', conn, if_exists='replace', index=False)
user.to_sql('user', conn, if_exists='replace', index=False)

96324

## Closed-End Questions

### What are the top 5 brands by receipts scanned among users 21 and over?

In [39]:
query1 = """
SELECT 
    p.BRAND 
  , COUNT(DISTINCT t.RECEIPT_ID) as num_receipts
FROM product as p 
JOIN "transaction" as t 
ON p.BARCODE = t.BARCODE
JOIN user as u 
ON t.USER_ID = u.ID
WHERE u.BIRTH_DATE <=  DATE('now', '-21 years') -- Users aged 21 and over
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
"""

top5_brand = pd.read_sql_query(query1, conn)
top5_brand

,BRAND,num_receipts
0,NERDS CANDY,3
1,DOVE,3
2,None,3
3,TRIDENT,2
4,SOUR PATCH KIDS,2


### What are the top 5 brands by sales among users that have had their account for at least six months?

In [40]:
query2 = """
SELECT
  p.BRAND
  , SUM(t.FINAL_SALE) AS total_sales
FROM "transaction" as t
JOIN product as p
ON t.BARCODE = p.BARCODE
JOIN user as u
ON t.USER_ID = u.ID
WHERE DATE('now', '-6 months') >= u.CREATED_DATE -- Users that have had their account for at least six months
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
"""


top5_brand_by_sales = pd.read_sql_query(query2, conn)
top5_brand_by_sales

,BRAND,total_sales
0,CVS,72.00
1,DOVE,30.91
2,TRIDENT,23.36
3,COORS LIGHT,17.48
4,TRESEMMÉ,14.58


### What is the percentage of sales in the Health & Wellness category by generation?

In [41]:
query3 = """
WITH cte AS (
SELECT
    u.ID
  , u.BIRTH_DATE
  , CASE
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1928 AND 1945 THEN 'Silent Generation'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1946 AND 1964 THEN 'Baby Boomers'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1965 AND 1980 THEN 'Gen X'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1981 AND 1996 THEN 'Millennials'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1997 AND 2012 THEN 'Gen Z'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) >= 2013 THEN 'Gen Alpha'
        ELSE 'Unknown'
    END AS generation
  , t.FINAL_SALE
FROM "transaction" as t
JOIN product as p
ON t.BARCODE = p.BARCODE
JOIN user as u
ON t.USER_ID = u.ID
WHERE p.CATEGORY_1 = 'Health & Wellness'
)
SELECT
    c.generation
  , ROUND(SUM(c.FINAL_SALE)*100 / (SELECT SUM(FINAL_SALE) FROM CTE), 2) AS health_wellness_percentage
FROM cte as c
GROUP BY 1
ORDER BY 2 DESC
"""


health_wellness_percentage = pd.read_sql_query(query3, conn)
health_wellness_percentage

,generation,health_wellness_percentage
0,Baby Boomers,54.26
1,Gen X,23.70
2,Millennials,22.04


## Open-Ended Questions 

### Who are Fetch’s power users?

Typically, a power user is someone who engages with the product more intensely than the average user. This could mean they make more transactions, use more features, or are more active over a certain period.

To define power users, we need to establish criteria based on user activity and engagement. Potential metrics could include:
- **Frequency of Transactions**: Number of transactions a user has made.

- **Total Spending**: The cumulative amount spent by a user.

- **Recency of Activity**: How recently a user has been active.

- **Product Diversity**: Number of different products a user has purchased.

For this case study, let's define a power user as someone who:
- Has spent more than $500 in total.

In [42]:
query11= """
SELECT 
    u.ID
  , SUM(t.FINAL_SALE) as total_spend
FROM user as u 
JOIN "transaction" as t 
ON u.ID = t.USER_ID
GROUP BY 1 
HAVING SUM(t.FINAL_SALE) > 500
ORDER BY 2 DESC
"""


power_user = pd.read_sql_query(query11, conn)
power_user

,ID,total_spend


### Which is the leading brand in the Dips & Salsa category?

The **leading brand** can be defined based on different metrics such as:

- **Total Sales Revenue**: The brand with the highest total sales amount in the "Dips & Salsa" category.

- **Number of Transactions**: The brand with the most transactions in the "Dips & Salsa" category.

- **Number of Unique Customers**: The brand with the most unique users purchasing their products in the category.

For this case study, let's define the leading brand as the one with the highest total sales revenue in the "Dips & Salsa" category.

In [43]:
query22 = """
WITH CTE AS (
SELECT
  p.BRAND
  , COUNT(DISTINCT t.USER_ID) AS unique_customers
  , DENSE_RANK() OVER(ORDER BY SUM(t.FINAL_SALE)) AS rnk
FROM "transaction" as t
JOIN product as p
ON t.BARCODE = p.BARCODE
WHERE p.CATEGORY_2 = 'Dips & Salsa'
GROUP BY 1
)

SELECT
  cte.BRAND
  , cte.unique_customers
FROM CTE
WHERE rnk=1
"""


leading_brand = pd.read_sql_query(query22, conn)
leading_brand

,BRAND,unique_customers
0,ZACCA,1


### At what percent has Fetch grown year over year?

To calculate Fetch's year-over-year (YoY) growth, we need to determine the percentage increase or decrease in a specific metric over two consecutive years. Common metrics for growth analysis include:

- **Total Revenue**: The sum of all transaction amounts.

- **Number of Transactions**: The total count of transactions.

- **Number of Active Users**: The count of unique users making transactions.

For this case study, assuming the current year is 2024 and the previous year is 2023; let's focus on total revenue as the metric for growth calculation.


Year-over-year revenue growth is calculated using the formula:

$
\text{YoY Revenue Growth} = \left( \frac{\text{Total Revenue in Current Year} - \text{Total Revenue in Previous Year}}{\text{Total Revenue in Previous Year}} \right) \times 100
$



In [ ]:
query33 = """
WITH cte AS (
SELECT
    strftime('%Y', t.SCAN_DATE) AS transaction_year
  , SUM(t.FINAL_SALE) AS total_revenue
FROM "transaction" t
WHERE strftime('%Y', t.SCAN_DATE) IN (2023, 2024)
GROUP BY 1
)
SELECT
    MAX(CASE WHEN transaction_year = 2024 THEN total_revenue END) AS current_year_revenue
  , MAX(CASE WHEN transaction_year = 2023 THEN total_revenue END) AS previous_year_revenue
  , (MAX(CASE WHEN transaction_year = 2024 THEN total_revenue END) - MAX(CASE WHEN transaction_year = 2023 THEN total_revenue END)) / MAX(CASE WHEN transaction_year = 2023 THEN total_revenue END) * 100 AS yoy_growth_percent
FROM
    cte
"""

yoy_revenue_growth = pd.read_sql_query(query33, conn)
yoy_revenue_growth

## Interesting Trend

One interesting trend in the data is the **dominance of CVS** among users who have had their accounts for at least six months. CVS leads significantly with **$72 in total sales**, far outpacing the next top brands:

- **DOVE**: \$30.91
- **TRIDENT**: \$23.36
- **COORS LIGHT**: \$17.48
- **TRESEMMÉ**: \$14.58

This suggests that **CVS is the most preferred brand** among long-term Fetch users, potentially due to its strong market presence, frequent promotions, or alignment with user preferences. The gap between CVS and the other brands highlights its strong performance in this user segment.

The trend underscores CVS's exceptional performance among long-term Fetch users and provides valuable insights for optimizing brand partnerships and user engagement strategies.